In [3]:
!pip install wandb
!pip install timm
import sys
import os
import gc
import random
import time
import math
import re
import string
import tqdm
import wandb
from PIL import Image
import glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD, AdamW
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold, StratifiedGroupKFold

from tqdm import tqdm
import torchvision
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=0517e6f3e47159a7a8a6bb9f3382ce96cd021339165746ce7fe12a270cf59089
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00


In [4]:
class CFG:
    DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
    DEBUG = False
    EXPERIMENT_NAME = '0'
    MODEL_NAME = 'efficientnet_b0'
    SEED= 42
    N_FOLDS = 10
    IMG_SIZE= 224
    IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
    IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB
    LEARNING_RATE= 1e-4
    WEIGHT_DECAY= 0
    T_MAX= 10
    T_0= 5
    ETA_MIN= 0
    TRAIN_BATCHSIZE= 70
    TEST_BATCHSIZE= 70
    EPOCHS = 15

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic= True

set_seed(CFG.SEED)

In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

#Import the libraries
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Rahul.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp/ADataset') #Extracts the files into the /tmp folder
zip_ref.close()

In [7]:
import pandas as pd

In [8]:
import pandas as pd
import glob
image_paths1=glob.glob('/tmp/ADataset/HAR/SITTING/*')
image_paths2=glob.glob('/tmp/ADataset/HAR/STANDING/*')
image_paths3=glob.glob('/tmp/ADataset/HAR/WALKING/*')
image_paths4=glob.glob('/tmp/ADataset/HAR/SLEEPING/*')
df_sit=pd.DataFrame({'image':image_paths1 , 'label':'sitting'})
df_stand=pd.DataFrame({'image':image_paths2 , 'label':'standing'})
df_walk=pd.DataFrame({'image':image_paths3 , 'label':'walking'})
df_sleep=pd.DataFrame({'image':image_paths4 , 'label':'sleeping'})
print(df_sit)
print(df_stand)
print(df_walk)
print(df_sleep)

                                                 image    label
0                 /tmp/ADataset/HAR/SITTING/img137.jpg  sitting
1                 /tmp/ADataset/HAR/SITTING/img340.jpg  sitting
2                 /tmp/ADataset/HAR/SITTING/img485.jpg  sitting
3                 /tmp/ADataset/HAR/SITTING/img222.jpg  sitting
4                 /tmp/ADataset/HAR/SITTING/img540.jpg  sitting
..                                                 ...      ...
445                /tmp/ADataset/HAR/SITTING/img45.jpg  sitting
446                /tmp/ADataset/HAR/SITTING/img16.jpg  sitting
447               /tmp/ADataset/HAR/SITTING/img563.jpg  sitting
448               /tmp/ADataset/HAR/SITTING/img165.jpg  sitting
449  /tmp/ADataset/HAR/SITTING/kavi-creation-pKN1aA...  sitting

[450 rows x 2 columns]
                                        image     label
0        /tmp/ADataset/HAR/STANDING/img77.jpg  standing
1       /tmp/ADataset/HAR/STANDING/img222.jpg  standing
2       /tmp/ADataset/HAR/STANDING/img54

In [9]:
df = pd.concat([df_sit ,df_stand,df_walk,df_sleep])

In [10]:
df.label.value_counts()

sitting     450
standing    450
walking     450
sleeping    450
Name: label, dtype: int64

In [11]:
df.shape

(1800, 2)

In [12]:
df['target']=df['label'].map({
    'standing' :'standing',
    'sitting':'sitting',
    'walking':'walking',
    'sleeping':'sleeping'
})

In [13]:
df.isnull().sum()

image     0
label     0
target    0
dtype: int64

In [14]:
df

,image,label,target
0,/tmp/ADataset/HAR/SITTING/img137.jpg,sitting,sitting
1,/tmp/ADataset/HAR/SITTING/img340.jpg,sitting,sitting
2,/tmp/ADataset/HAR/SITTING/img485.jpg,sitting,sitting
3,/tmp/ADataset/HAR/SITTING/img222.jpg,sitting,sitting
4,/tmp/ADataset/HAR/SITTING/img540.jpg,sitting,sitting
...,...,...,...
445,/tmp/ADataset/HAR/SLEEPING/img563.jpg,sleeping,sleeping
446,/tmp/ADataset/HAR/SLEEPING/img32 (2).jpg,sleeping,sleeping
447,/tmp/ADataset/HAR/SLEEPING/img535.jpg,sleeping,sleeping
448,/tmp/ADataset/HAR/SLEEPING/img96 (2).jpg,sleeping,sleeping


In [15]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=CFG.SEED, shuffle=True, stratify=df['label'])

In [16]:
train_transforms = A.Compose([
    A.Resize(width= CFG.IMG_SIZE, height= CFG.IMG_SIZE, p=1),
    A.Normalize(mean=CFG.IMAGENET_MEAN, std=CFG.IMAGENET_STD, p=1.0),
    ToTensorV2(p=1.0)
])

valid_transforms = A.Compose([
    A.Resize(width= CFG.IMG_SIZE, height= CFG.IMG_SIZE, p=1.0),
    A.Normalize(mean=CFG.IMAGENET_MEAN, std=CFG.IMAGENET_STD, p=1.0),
    ToTensorV2(p=1.0)
])


class CustomDataset:
    def __init__(self, df, augmentations= None):
        self.df = df
        self.augmentations = augmentations

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
#         print(self.image_ids)
        image_loc = self.df.image[idx]
#         print(image_id)
        image = cv2.cvtColor(cv2.imread(image_loc), cv2.COLOR_BGR2RGB)

        if self.augmentations:
            image = self.augmentations(image=image)['image']

        target = self.df.target[idx]

        return image, target

In [17]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.backbone = timm.create_model(CFG.MODEL_NAME, pretrained=True)
        self.input_dim = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Identity()
#         self.dropout = nn.Dropout(p = 0.3)
        self.fc = nn.Linear(self.input_dim, 3)
#         nn.init.xavier_normal_(self.fc.weight)

    def forward(self, image):
        features = self.backbone(image)
        features = features.reshape(features.shape[0], -1)
        output = self.fc(features)
#         output = self.fc(self.dropout(features))
        return output

In [18]:
def train_model(dataloader, model, optimizer, criterion):
    model.train()
    n_batches= len(dataloader)

    train_loss= 0

    for images, targets in tqdm(dataloader):

        images= images.to(CFG.DEVICE)
        targets = targets.to(CFG.DEVICE)
        output = model(images)
        loss = criterion(output, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss

    return train_loss/n_batches


def val_model(dataloader, model, criterion):
    model.eval()
    n_batches= len(dataloader)
    val_loss= 0
    preds= np.array([])

    for images, targets in tqdm(dataloader):
        images= images.to(CFG.DEVICE)
        targets = targets.to(CFG.DEVICE)

        with torch.no_grad():
            output = model(images)

        loss = criterion(output, targets)
        val_loss += loss
        softmax = nn.Softmax(dim=1)
        output= softmax(output).cpu().detach().numpy()
        output = np.argmax(output, axis=1)
        preds = np.concatenate((preds, output))

    return val_loss/n_batches, preds

In [1]:
import os
directory='/tmp/working'
if not os.path.exists(directory):
  os.makedirs(directory)
train_dataset = CustomDataset(train_df.reset_index(drop=True), augmentations= train_transforms)
val_dataset = CustomDataset(val_df.reset_index(drop=True), augmentations= valid_transforms)

train_loader = DataLoader(train_dataset, batch_size= CFG.TRAIN_BATCHSIZE, shuffle=True,  pin_memory=2)
val_loader = DataLoader(val_dataset, batch_size= CFG.TEST_BATCHSIZE, shuffle=False, pin_memory=2)

model = CustomModel().to(CFG.DEVICE)
criterion = nn.CrossEntropyLoss()

optimizer = AdamW(model.parameters(), lr=CFG.LEARNING_RATE)

best_val_loss = float('inf')

for i in range(CFG.EPOCHS):
    avg_train_loss = train_model(train_loader, model, optimizer, criterion)
    avg_val_loss, preds = val_model(val_loader, model, criterion)

    print(f'Epoch: {i}, train_loss: {avg_train_loss}, val_loss: {avg_val_loss}')

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), f'/tmp/working/model_{i}.pt')
        val_df['preds'] = preds

del model
torch.cuda.empty_cache()


NameError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(val_df['target'], val_df['preds'])
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

In [ ]:
temp_df=val_df[val_df['target']!=val_df['preds']].reset_index(drop=True)

In [ ]:
temp_df['preds_label']=temp_df['preds'].map({
0:'standing',
1:'sitting',
2:'walking'

})

In [ ]:
temp_df

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(val_df['target'],val_df['preds'])
print(f'Accuracy:{accuracy*100:2f}%')

In [ ]:
# Count of wrongly predicted samples
wrongly_predicted_count = len(temp_df)

# Count of total samples
total_samples_count = len(val_df)
print(wrongly_predicted_count)
print(total_samples_count)
# Calculate accuracy
accuracy = 1.0 - (wrongly_predicted_count / total_samples_count)

# Print accuracy as a percentage
print(f'Accuracy: {accuracy * 100:.2f}%')